In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
# from langchain.document_loaders import UnstructuredFileLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=1500,
    chunk_overlap=300
)
loader = TextLoader("./files/animal_farm.txt")

# docs = loader.load()
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vector_store = FAISS.from_documents(docs, cached_embeddings)

retriver = vector_store.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer, just say you don't know. Don't make it up:\n\n{context}"
    ),
    ("human", "{question}")
])

chain = { "context": retriver, "question": RunnablePassthrough() } | prompt | llm

chain.invoke("Describe leader of Animal Farm")
# chain.run("What is the name of the book?")

AIMessage(content='The leader of Animal Farm is Napoleon, who is a pig. He is described as taking on a heavy responsibility and is in control of the farm, along with his colleagues. Napoleon is shown to make decisions and changes on the farm, including suppressing certain customs, promoting confidence, and changing the name of the farm to "The Manor Farm."')